# Shock Tube: Unsteady Euler Simulation
This workbook contains a one-dimensional inviscid model of a shock tube.  These tubes initially consist of two sections: a high pressure driver section and a low pressure driven section.



In [7]:
"""Import Python modules used in simulation"""
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
"""Define the state vector class"""

class State:
    """The State class is used to store the state (and composition) vector at a single grid cell: 
        1. ρ: Density (dimensionless)
        2. u: Velocity (dimensionless)
        3. Et: Total Energy (dimensionless)
        4. γ: Ratio of specific heats (dimensionless)
        """
    def __init__(self, ρ=None, u=None, Et=None, ga=None):        
        self.ρ = ρ    # Dimensionless Density
        self.u = u    # Dimensionless Velocity
        self.Et = Et  # Dimensionless Total Energy
        self.γ = γ    # Ratio of specific heats (private)
        
    @property
    def a(self):
        """a: Speed of sound (dimensionless)"""
        return np.sqrt( self.γ*(self.γ-1) * (self.Et/self.ρ - self.u**2/2) )
        
    @property
    def p(self):
        """p: Static pressure (dimensionless)"""
        return self.ρ*self.a**2/self.γ;

In [17]:
"""Define the grid class"""

class Grid:
    """The Grid class generat"""
    
    def __init__(self, N_points=100, L_driver=1):
        """Initialize the class by generating a grid with an initial distribution of uniformly 
            spaced grid points (cell edges).  The two (optional) input parameters are
            1. N_points: The number of (cell edge) grid points with a default of 100, and
            2. L_driver: The length of the driver section of the tube (dimensionless)"""
        self.N_points = N_points
        self.L_driver = L_driver
        self.L_tube = 1 + L_driver
        self.UniformGrid = np.linspace(0, 1 + L_driver, N_points)
        self.ActiveGrid = self.UniformGrid
        
    def NonUniformGrid(self, Ψorig):
        """Generate a nonuniform grid whose spacing is determined by gradients in the flow parameter
            (e.g. density) provided."""
    
        # Map the flow parameter onto a uniform grid via interpolation
        Ψuniform = np.interp(self.UniformGrid, self.ActiveGrid, Ψorig)
        
        """Calculate the absolute value of the change in density across each grid cell but set
        a floor that is equal to 1/10 of the difference between the maximum and minimum Ψorig"""
        ΔΨ = np.maximum(np.abs(np.max(Ψorig)-np.min(Ψorig))/10, 
                        np.abs(np.diff(Ψorig)))
        

        if np.sum(ΔΨ) == 0:
            x_new = self.UniformGrid
            
        else:
            """Calculate the new distance between grid points"""
            Δx_new = np.maximum(np.diff(self.ActiveGrid), 
                               self.L_tube/10/self.N_points) / ΔΨ * np.mean(Ψorig)

            # Generate the new grid        
            x_new = np.concatenate( (np.zeros(1), np.cumsum(Δx_new)) )
            x_new = x_new * self.L_tube / ( x_new[-1] - x_new[0] )
        
        # Save it to ActiveGrid
        return x_new

In [18]:
x = Grid(100,1)
display(x)

In [19]:
display(x.UniformGrid)
display(x.NonUniformGrid(np.ones(100)))

array([ 0.        ,  0.02020202,  0.04040404,  0.06060606,  0.08080808,
        0.1010101 ,  0.12121212,  0.14141414,  0.16161616,  0.18181818,
        0.2020202 ,  0.22222222,  0.24242424,  0.26262626,  0.28282828,
        0.3030303 ,  0.32323232,  0.34343434,  0.36363636,  0.38383838,
        0.4040404 ,  0.42424242,  0.44444444,  0.46464646,  0.48484848,
        0.50505051,  0.52525253,  0.54545455,  0.56565657,  0.58585859,
        0.60606061,  0.62626263,  0.64646465,  0.66666667,  0.68686869,
        0.70707071,  0.72727273,  0.74747475,  0.76767677,  0.78787879,
        0.80808081,  0.82828283,  0.84848485,  0.86868687,  0.88888889,
        0.90909091,  0.92929293,  0.94949495,  0.96969697,  0.98989899,
        1.01010101,  1.03030303,  1.05050505,  1.07070707,  1.09090909,
        1.11111111,  1.13131313,  1.15151515,  1.17171717,  1.19191919,
        1.21212121,  1.23232323,  1.25252525,  1.27272727,  1.29292929,
        1.31313131,  1.33333333,  1.35353535,  1.37373737,  1.39

/Users/david.tew/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in true_divide
/Users/david.tew/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


array([  0.,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan])